# ID2214/FID3214 Assignment 3 Group no. 4
### Project members: 
Lucas Larsson, lulars@kth.se
Mihaela Bakšić, baksic@kth.se

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [40]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier
from functools import reduce

In [2]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.9.13
NumPy version: 1.23.3
Pandas version: 1.4.4
sklearn version: 1.0.2


## Reused functions from Assignment 1

In [3]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_column_filter(df):
    filter = []
    df_filtered = df.copy()
    for (col_name, col_data) in df_filtered.iteritems():
        if col_name in ['CLASS', 'ID'] or not df_filtered[col_name].dropna().nunique() <= 1:
            filter.append(col_name)
    
    return df_filtered[filter], filter

def apply_column_filter(df, filter):
    df_filtered = df.copy()[filter]
    return df_filtered


def create_imputation(df):
    df_imp = df.copy()
    imputation = dict()

    for (col_name, col_data) in df_imp.select_dtypes(["int", 'float']).iteritems():
        if col_name not in ['CLASS', 'ID']:
            mean_col = df_imp[col_name].mean() if df_imp[col_name].count() != 0 else 0
            df_imp[col_name].fillna(mean_col, inplace=True)
            
            imputation[col_name] = mean_col
        
    for (col_name, col_data) in df_imp.select_dtypes(["object", 'category']).iteritems():
        if col_name not in ['CLASS', 'ID']:
            mode_col = df_imp[col_name].mode()[0]
            df_imp[col_name].fillna(mode_col, inplace=True)
            
            imputation[col_name] = mode_col
        
    return df_imp, imputation

    

def apply_imputation(df, imputation):
    df_imp = df.copy()

    for col_name in imputation:
        df_imp.fillna(imputation[col_name], inplace=True)

    return df_imp

def create_one_hot(df):
    copied_df = df.copy()
    second_copied_df = df.copy()
    one_hot_mapping = dict()

    excluded_columns = ['ID', 'CLASS']
    columns = list(df.select_dtypes(include=['object','category']))
    columns = [i for i in columns if i not in excluded_columns]

    for col in columns:
        copied_df[col] = copied_df[col].astype('category')
        one_hot_mapping[col] = list(copied_df[col].cat.categories)

        for i in one_hot_mapping[col]:
            col_name = col +'_'+ i
            new_col = copied_df[col].apply(lambda x: 1.0 if x == i else 0.0)
            second_copied_df[col_name] = new_col

        second_copied_df = second_copied_df.drop(columns = col, axis = 1)

    return second_copied_df, one_hot_mapping


def apply_one_hot(df, mapping):
    copied_df = df.copy()
    second_copied_df = df.copy()

    for col in copied_df.columns:
        if col in mapping.keys():
            for i in mapping[col]:
                col_name = col +'_'+ i
                new_col = copied_df[col].apply(lambda x: 1.0 if x == i else 0.0)
                second_copied_df[col_name] = new_col

            second_copied_df = second_copied_df.drop(columns = col, axis = 1)

    return second_copied_df


def accuracy(dataframe, correctlabels):

    df = dataframe.copy()

    labels_df = df.idxmax(axis=1)
    truelabels_df = (labels_df == correctlabels).sum(axis=0)
    accuracy_df = truelabels_df/len(df)

    return accuracy_df


def brier_score(dataframe, correctlabels):

    df = dataframe.copy()

    cor_df = pd.get_dummies(correctlabels)
    brierscore = np.mean(np.sum((df - cor_df)**2, axis=1))

    return brierscore


def get_roc(predictions, correct_labels, score, index):

    crct = np.array(correct_labels)==predictions.columns[index]
    not_crct = np.array(correct_labels)!=predictions.columns[index]

    pred = predictions[predictions.columns[index]]

    tp = sum(pred[crct] >= score) / sum(crct)
    fp = sum(pred[not_crct] >= score) / sum(not_crct)

    return tp, fp


def auc(predictions, correctlabels):
    scores = np.unique(predictions)[::-1]
    nmbr_tot_labels = len(correctlabels)

    AUCs = {}

    for (idx,c) in enumerate(np.unique(correctlabels)):
        roc_pointes = [get_roc(predictions, correctlabels, score, idx) for score in scores]

        AUC = 0
        prev_tp = 0
        prev_fp = 0

        # we kept having problem with calculating AUC using the algorithm provided in the lecture slides.
        # So we just calculate the triangle and rectangles using the respective method to calculate
        # - tringle area = (height  * base) / 2
        # - rectangle area = len * wid
        for point in roc_pointes:
            tp, fp = point

            if tp > prev_tp and fp > prev_fp:
                AUC += (fp-prev_fp)*prev_tp + (fp-prev_fp)*(tp-prev_tp) / 2

            elif fp > prev_fp:
                AUC += (fp-prev_fp)*tp

            prev_tp = tp
            prev_fp = fp

        AUCs[c] = AUC

    AUC_tot = 0
    for col, auc in AUCs.items():
        AUC_tot +=  np.sum(np.array(correctlabels) == col) / nmbr_tot_labels * auc

    return AUC_tot




## 1. Define the class RandomForest

In [81]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 


class RandomForest():

    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):

        df_original = df.copy()

        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)

        self.labels = df_original['CLASS']

        df.drop(columns=['CLASS'], inplace=True)
        
        # generate models
        self.model = [ DecisionTreeClassifier(max_features='log2', max_depth=10) for i in range(no_trees)] 

        # fit models
        for tree in self.model:
            subsample_idx = np.random.choice(df.index.tolist(), size=100, replace=True)
            tree.fit(X=df.iloc[subsample_idx], y=self.labels.iloc[subsample_idx])



    def predict(self, df):
        
        df_original = df.copy()

        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df, self.one_hot)

        labels = df['CLASS']

        df.drop(columns=['CLASS'], inplace=True)
        
        
        tree_predictions_accumulated = [pd.DataFrame(tree.predict_proba(X = df), columns=tree.classes_) for  tree in self.model]

        df_probabilities = reduce(lambda a, b: a.add(b, fill_value=0), tree_predictions_accumulated)

        return df_probabilities.div(len(self.model))






In [82]:
train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

rf.fit(train_df, 100)

rf.predict(train_df)


,negative,positive
0,0.610000,0.390000
1,0.420000,0.580000
2,0.760000,0.240000
3,0.180000,0.820000
4,0.258333,0.741667
...,...,...
474,0.085833,0.914167
475,0.566667,0.433333
476,0.330000,0.670000
477,0.370000,0.630000


In [80]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.48 s.
Testing time: 0.33 s.
Accuracy: 0.7432
AUC: 0.8999
Brier score: 0.3315


In [83]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 0.9395
AUC on training set: 0.9986
Brier score on training set: 0.1704


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [40]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to


class RandomForestNonAligned(RandomForest):

    def predict(self, df):

        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df, self.one_hot)

        labels = df['CLASS']
        possible_labels = sorted(set(labels))
        print(possible_labels)
        df.drop(columns=['CLASS'], inplace=True)

        tree_predictions = [pd.DataFrame(tree.predict_proba(X = df), columns=tree.classes_) for  tree in self.model]
        for i in tree_predictions:
            for label in possible_labels:
                if label not in i.columns:
                    i[label] = np.zeros(len(df))

        df_probabilities = reduce(lambda a, b: a.add(b, fill_value=0), tree_predictions)

        return df_probabilities.div(len(self.model))



In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.15 s.
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9741
Brier score: 0.0975


## 2b. Estimate predictive performance using out-of-bag predictions

In [8]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0

In [9]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 1.96 s.
OOB accuracy: 0.9555
Testing time: 0.06 s.
Accuracy: 0.9488
AUC: 0.9718
Brier score: 0.0986


In [10]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.